## Student Information
Please fill in the block below. End each line with two spaces.

Name: Yu Huai  
Email: yhuai@brandeis.edu   
Date: 11/17/2018 

In [5]:
import pandas as pd

In [4]:
# Sample data path is used for display.
sample_data_path = 'Online_Retail_Sample.csv'

# You may wish to extract other samples from the full dataset
# for practice.
real_data_path = 'Online_Retail.csv'

# Sample Data Format

In [85]:
# The general format of the dataset.
# Each row corresponds to a transaction.
# Each number corresponds to the ID of a product bought in the transaction.
df = pd.read_csv(sample_data_path, names=list(range(0,6)))
display(df)

,0,1,2,3,4,5
0,1084,1097,1126,2183.0,2375.0,NaN
1,1261,1394,2375,NaN,NaN,NaN
2,582,644,668,1082.0,1100.0,NaN
3,349,897,1142,1243.0,2316.0,2363.0
4,1098,1143,1816,2375.0,2402.0,NaN
5,121,219,363,1500.0,1943.0,NaN
6,964,1017,1126,2096.0,2183.0,NaN
7,1079,1189,2316,2356.0,NaN,NaN
8,766,1079,1720,1816.0,2356.0,NaN
9,209,298,593,1565.0,NaN,NaN


# Expected Output Format

#### Helper functions for prepping dummy data

In [ ]:
def generate_dummy_data():
    item_a_id = pd.DataFrame(random.sample(range(1, 2000), 100))
    item_b_id = pd.DataFrame(random.sample(range(1, 2000), 100))
    support_ab = pd.DataFrame([random.uniform(0, 0.1) for _ in range(100)])
    confidence_a_to_b = pd.DataFrame([random.uniform(0, 0.5) for _ in range(100)])
    confidence_b_to_a = pd.DataFrame([random.uniform(0, 0.5) for _ in range(100)])
    return [item_a_id, item_b_id, support_ab,
            confidence_a_to_b, confidence_b_to_a]

def generate_cols():
    cols = ['item_a',
            'item_b',
            'support_ab',
            'confidence_a_to_b',
            'confidence_b_to_a']
    return cols

def prep_df(dummy_data, cols):
    df = pd.concat(dummy_data, axis=1)
    df.columns = cols
    df.sort_values(by='support_ab', inplace=True, ascending=False)
    df.reset_index(inplace=True, drop=True)
    return df

### Expected output

Below you may find the type of output that you are expected to produce, sorted by support.

In [ ]:
import pandas as pd
import random

dummy_data = generate_dummy_data()
cols = generate_cols()

df = prep_df(dummy_data, cols)

display(df)

# Your code

You may use any existing packages, algorithms, etc.  

Please include website links for any imports used.  

Please describe your code and how you used any imports in the comments.  

In [ ]:
# Your code goes here.

In [ ]:
import pandas as pd
# from IPython import embed
real_data_path = 'Online_Retail.csv'
df = pd.read_csv(real_data_path, names=list(range(0,6)))
data = df.values
# test_data = data[:1000]
# test_df = df.iloc[:1000]

def generateTwoItemsets(oneItemsets, miniSupport, itemFreq):
    ## if 1-itemset is not frequent(< miniSupport), skip it
    ## 比如 oneItemsets = [a,b,c,d], 但是a 出现次数< minisupport, return [(b,c),(b,d),(c,d)]
    num = len(oneItemsets)
    twoItemsets = []
    for i in range(num-1):
        item1 = oneItemsets[i]
        if itemFreq[item1] < miniSupport:
            continue
        for j in range(i+1, num):
            item2 = oneItemsets[j]
            if itemFreq[item2] < miniSupport:
                continue
            twoItemsets.append((item1, item2))
    return twoItemsets

def oneItemsetFreq(data):
    #count item occurance
    dataFlat = data.flatten()
    import collections as cl
    itemFreq = cl.Counter(dataFlat)
    return itemFreq

def twoItemsetFreq(df, itemFreq, miniSupport):
    ## 遍历每一条数据， 统计pair出现次数
    total, _ = df.shape
    twoItemFreq = {}
    for i in range(total):
        items = list(df.iloc[i])
        pairs = generateTwoItemsets(items, miniSupport, itemFreq)
        for pair in pairs:
            item1, item2 = pair
            twoItemFreq[(item1, item2)] = twoItemFreq.get((item1, item2), 0) + 1
    return twoItemFreq
    

def computeConfidence(itemFreq, twoItemsets, item1, item2):
    confidence1 = twoItemsets[(item1, item2)] / itemFreq[item1]
    confidence2 = twoItemsets[(item1, item2)] / itemFreq[item2]
    return confidence1, confidence2
    
total, _ = df.shape    
miniSupport = 50  ##我设了一个threshold过滤掉那些单个出现频率特别少的， 50是我随便写的 
oneItemFreq = oneItemsetFreq(data)  ## 统计每个item出现的次数
# oneItemFreq = oneItemsetFreq(test_data)
twoItemsets = twoItemsetFreq(df, oneItemFreq, miniSupport)  ##统计data里的每一个pair出现的次数， 如果a出现< minisupport，就不统计和a组队的了
item_a = []
item_b = []
support_ab = []
confidence_a_to_b = []
confidence_b_to_a = []
for pair, support in twoItemsets.items():
    itema, itemb = pair
    item_a.append(itema)
    item_b.append(itemb)
    support_ab.append(support/ total * 1.0)
    conf1, conf2 = computeConfidence(oneItemFreq, twoItemsets, itema, itemb)
    confidence_a_to_b.append(conf1)
    confidence_b_to_a.append(conf2)

cols = generate_cols()
outdata = [pd.DataFrame(item_a), pd.DataFrame(item_b),pd.DataFrame(support_ab), pd.DataFrame(confidence_a_to_b),pd.DataFrame(confidence_b_to_a) ]
output =  prep_df(outdata, cols)
output.head(100)